In [1]:
import numpy as np
import pandas as pd
import sys
import sklearn
from sklearn import preprocessing 
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import cudf
import cupy
import psutil 
import gc
from cuml.ensemble import RandomForestClassifier as curfc
from sklearn.ensemble import RandomForestClassifier as skrfc
from cuml import make_regression
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression
from cuml.neighbors import KNeighborsClassifier as KNeighborsC
from sklearn.neighbors import KNeighborsClassifier
from cuml.linear_model import MBSGDClassifier as cumlMBSGDClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from cuml import Ridge
from cuml.linear_model import Ridge
from sklearn.linear_model import Ridge
from cuml import LogisticRegression
from sklearn.linear_model import LogisticRegression as skLogistic
from cuml.linear_model import ElasticNet
from sklearn import linear_model
from cuml.linear_model import Lasso
from cuml.solvers import SGD as cumlSGD

In [2]:
%time df = pd.read_csv('data_proc.csv')
print(df)

CPU times: user 7.65 s, sys: 1.12 s, total: 8.77 s
Wall time: 13.4 s
         Severity  Start_Lat   Start_Lng    End_Lat     End_Lng  Distance(mi)  \
0               3  40.108910  -83.092860  40.112060  -83.031870         3.230   
1               2  39.865420  -84.062800  39.865010  -84.048730         0.747   
2               2  39.102660  -84.524680  39.102090  -84.523960         0.055   
3               2  41.062130  -81.537840  41.062170  -81.535470         0.123   
4               3  39.172393  -84.492792  39.170476  -84.501798         0.500   
...           ...        ...         ...        ...         ...           ...   
2845336         2  34.239104 -118.416176  34.239104 -118.416176         0.000   
2845337         2  34.002480 -117.379360  33.998880 -117.370940         0.543   
2845338         2  32.766960 -117.148060  32.765550 -117.153630         0.338   
2845339         2  33.775450 -117.847790  33.777400 -117.857270         0.561   
2845340         2  33.992460 -118.403020

In [3]:
df.head()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),County,State,Temperature(F),Humidity(%),...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,out
0,3,40.108910,-83.092860,40.112060,-83.031870,3.230,Franklin,OH,42.1,58.0,...,False,False,False,False,False,Night,Night,Night,Night,5.198541
1,2,39.865420,-84.062800,39.865010,-84.048730,0.747,Montgomery,OH,36.9,91.0,...,False,False,False,False,False,Night,Night,Night,Night,1.201714
2,2,39.102660,-84.524680,39.102090,-84.523960,0.055,Hamilton,OH,36.0,97.0,...,False,False,False,False,False,Night,Night,Night,Day,0.088753
3,2,41.062130,-81.537840,41.062170,-81.535470,0.123,Summit,OH,39.0,55.0,...,False,False,False,False,False,Night,Night,Day,Day,0.198752
4,3,39.172393,-84.492792,39.170476,-84.501798,0.500,Hamilton,OH,37.0,93.0,...,False,False,False,False,False,Day,Day,Day,Day,0.805093


In [4]:
df = df.dropna()

In [5]:
df['Severity'].value_counts()

2    2532990
3     155105
4     131193
1      26053
Name: Severity, dtype: int64

In [6]:
psutil.virtual_memory().available * 100 / psutil.virtual_memory().total

49.815534928160844

In [7]:
# Separate majority and minority classes
df_s1 = df[df['Severity']==1]
df_s2 = df[df['Severity']==2]
df_s3 = df[df['Severity']==3]
df_s4 = df[df['Severity']==4]

In [8]:
count = min(df_s1.count()[0], df_s2.count()[0], df_s3.count()[0], df_s4.count()[0])

In [10]:
df_s1 = resample(df_s1, replace=df_s1.count()[0]<count, n_samples=count, random_state=42)

In [11]:
df_s2 = resample(df_s2, replace=df_s2.count()[0]<count, n_samples=count, random_state=42)

In [12]:
df_s3 = resample(df_s3, replace=df_s3.count()[0]<count, n_samples=count, random_state=42)

In [13]:
df_s4 = resample(df_s4, replace=df_s4.count()[0]<count, n_samples=count, random_state=42)

In [14]:
df_s12 = pd.concat([df_s1, df_s2])

In [15]:
df_s34 = pd.concat([df_s3, df_s4])

In [17]:
df = pd.concat([df_s12, df_s34])

In [19]:
df.groupby(by='Severity')['Severity'].count()

Severity
1    26053
2    26053
3    26053
4    26053
Name: Severity, dtype: int64

In [20]:
%%time
# Set the target for the prediction
target='Severity' 
cols = df.select_dtypes(include='object').columns

# set X and y
y = df[target]
X = df.drop(target, axis=1)

# Create the encoder.
encoder = OrdinalEncoder()
X[cols] = encoder.fit_transform(X[cols])

CPU times: user 211 ms, sys: 10.4 ms, total: 221 ms
Wall time: 216 ms


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size = 0.3, random_state=0)

In [23]:
%%time
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

CPU times: user 119 ms, sys: 59.7 ms, total: 178 ms
Wall time: 175 ms


KNeighborsClassifier(n_neighbors=3)

In [24]:
%%time
print(neigh.score(X_test, y_test))

0.5449078812691914
CPU times: user 2min 10s, sys: 3min 42s, total: 5min 52s
Wall time: 1min 8s


In [25]:
%%time
#Convert the data to CuDF dataframes here
X_cudf_train = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)

y_cudf_train = cudf.Series(y_train.values)
y_cudf_test = cudf.Series(y_test.values)

CPU times: user 3.93 s, sys: 1.78 s, total: 5.7 s
Wall time: 7.13 s


In [26]:
%%time
knn = KNeighborsC(n_neighbors=10)
knn.fit(X_cudf_train, y_cudf_train)

CPU times: user 356 ms, sys: 79.5 ms, total: 435 ms
Wall time: 837 ms


KNeighborsClassifier()

In [27]:
%%time
print(knn.score(X_cudf_test, y_test))

0.556006908416748
CPU times: user 471 ms, sys: 9.66 ms, total: 480 ms
Wall time: 540 ms
